## Silver: Processamento Viniferas

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:
%sql
select * from vitivinicultura.bronze_processamento.processaviniferas

In [0]:

df_original = spark.table("vitivinicultura.bronze_processamento.processaviniferas").toPandas()

# 2️⃣ Define a coluna com os dados separados por ";"
coluna = "_c0"

# 3️⃣ Colunas adicionais
colunas_adicionais = ["ingestao_ts", "usuario_ingestao"]
df_extra = df_original[colunas_adicionais].copy()

# 4️⃣ Quebra a string em colunas separadas
linhas = df_original[coluna].str.split(';', expand=True)

# 5️⃣ Garante que a primeira linha seja uma lista de strings válida
cabecalho = linhas.iloc[0].astype(str).fillna("coluna_sem_nome").tolist()

# 6️⃣ Define os nomes das colunas
linhas.columns = cabecalho

# 7️⃣ Remove a primeira linha (que era o cabeçalho original)
df_original = spark.table("vitivinicultura.bronze_processamento.processaviniferas").toPandas()

# 2️⃣ Define a coluna com os dados separados por ";"
coluna = "_c0"

# 3️⃣ Colunas adicionais
colunas_adicionais = ["ingestao_ts", "usuario_ingestao"]
df_extra = df_original[colunas_adicionais].copy()

# 4️⃣ Quebra a string em colunas separadas
linhas = df_original[coluna].str.split(';', expand=True)

# 5️⃣ Garante que a primeira linha seja uma lista de strings válida
cabecalho = linhas.iloc[0].astype(str).fillna("coluna_sem_nome").tolist()

# 6️⃣ Define os nomes das colunas
linhas.columns = cabecalho

# 7️⃣ Remove a primeira linha (que era o cabeçalho original)
df_tratado = linhas.iloc[1:].reset_index(drop=True)

# 8️⃣ Corrige nomes vazios e duplicados
df_tratado.columns = [
    f"{c}_{i}" if list(linhas.columns).count(c) > 1 else c
    for i, c in enumerate(linhas.columns)
]

# 8️⃣ Colunas que NÃO devem ser somadas e DEVEM permanecer como texto
nao_somar = ["id", "control","cultivar"]

# 9️⃣ Força as colunas de nao_somar para string
for c in df_tratado.columns:
    if any(c.startswith(ex) for ex in nao_somar):
        df_tratado[c] = df_tratado[c].astype(str).fillna("")
    else:
        # Converte as demais colunas para número se possível
        df_tratado[c] = pd.to_numeric(df_tratado[c], errors="coerce")

# 🔟 Separa colunas numéricas e texto
colunas_texto = [c for c in df_tratado.columns if any(c.startswith(ex) for ex in nao_somar)]
colunas_numericas = [c for c in df_tratado.columns if c not in colunas_texto]

# 11️⃣ Soma apenas colunas numéricas com nomes duplicados
df_num = df_tratado[colunas_numericas].groupby(lambda x: x.split("_")[0], axis=1).sum(min_count=1)

# 12️⃣ Garante que texto não seja afetado por agrupamento
df_texto = df_tratado[colunas_texto].copy()

# 13️⃣ Junta novamente texto e numéricos
df_tratado_final = pd.concat([df_texto, df_num], axis=1)

# 14️⃣ Junta as colunas adicionais
df_final = pd.concat([df_extra.reset_index(drop=True), df_tratado_final], axis=1)
df = df_final[df_final["id"].notnull()]

# 🔹 Visualiza o resultado
display(df)

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.silver_processamento.processaviniferas
USING DELTA
""")
df_spark = spark.createDataFrame(df_final)
# Escrita no formato Delta Lake
df_spark.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.silver_processamento.processaviniferas")

